In [1]:
import pandas as pd
import os
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta
from scipy.stats import pearsonr
from sklearn.manifold import TSNE
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import LeavePGroupsOut
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import seaborn as sns
from StudentLife import get_data

# display all columns and rows
pd.set_option('display.max_colwidth',-1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
NUM_DAYS_USED = 14
NUM_DAYS_BLOCKED = 1 # blocking the current ema day

behavior_names = [
                  'calllog',
                  'sms_in',
                  'sms_out',
                  'unlock',
                  'conver',
                  'sleep',
                  'act_invehicle',
                  'act_onbike',
                  'act_onfoot',
                  'act_still',
                  'act_tilting',
                  'act_unknown',
                  'light',
                  'audio_amp',
                  'audio_noise',
                  'audio_silence',
                  'audio_voice'                  
                 ]

# add previous ema

In [3]:
ema = pd.read_csv('table_ema_'+str(NUM_DAYS_USED)+'_'+str(NUM_DAYS_BLOCKED)+'.csv')
ema = ema.drop(['Unnamed: 0'], axis=1)
ema['day'] = pd.to_datetime(ema['day'])
print('num of ema: ', len(ema))
print('num of columns: ', len(ema.columns))

num of ema:  5027
num of columns:  118


In [4]:
# use previous ema exactly "NUM_DAY_MINUS" days ago
# change the commented out line of code to replace nan with mean
NUM_DAY_MINUS = 7
EMA_ITEM = 'ema_score'
users = [
'u004_rct@eureka',
'u005_rct@eureka',
'u006_rct@eureka',
'u007_rct@eureka',
'u008_rct@eureka',
'u009_rct@eureka',
'u010_rct@eureka',
'u011_rct@eureka',
'u012_rct@eureka',
'u013_rct@eureka',
'u014_rct@eureka',
'u015_rct@eureka',
'u016_rct@eureka',
'u017_rct@eureka',
'u018_rct@eureka',
'u019_rct@eureka',
'u020_rct@eureka',
'u021_rct@eureka',
'u022_rct@eureka',
'u023_rct@eureka',
'u024_rct@eureka',
'u025_rct@eureka',
'u026_rct@eureka',
'u027_rct@eureka',
'u028_rct@eureka',
'u029_rct@eureka',
'u030_rct@eureka',
'u031_rct@eureka',
'u032_rct@eureka',
'u033_rct@eureka',
'u034_rct@eureka',
'u035_rct@eureka',
'u036_rct@eureka',
'u037_rct@eureka',
'u038_rct@eureka',
'u039_rct@eureka',
'u040_rct@eureka',
'u041_rct@eureka',
'u042_rct@eureka',
'u043_rct@eureka',
'u044_rct@eureka',
'u045_rct@eureka',
'u046_rct@eureka',
'u047_rct@eureka',
'u048_rct@eureka',
'u049_rct@eureka',
'u050_rct@eureka',
'u051_rct@eureka',
'u052_rct@eureka',
'u053_rct@eureka',
'u054_rct@eureka',
'u055_rct@eureka',
'u056_rct@eureka',
'u057_rct@eureka',
'u058_rct@eureka',
'u059_rct@eureka',
'u060_rct@eureka',
'u061_rct@eureka',
'u062_rct@eureka',
'u063_rct@eureka',
'u064_rct@eureka',
'u065_rct@eureka',
'u066_rct@eureka',
'u067_rct@eureka',
'u068_rct@eureka',
'u069_rct@eureka',
'u070_rct@eureka',
'u071_rct@eureka',
'u072_rct@eureka',
'u073_rct@eureka',
]
print(len(users))
new_ema = pd.DataFrame()
for user in users:
    ema_user = ema.loc[ema.eureka_id == user[0:4]]
    ema_user = ema_user.set_index('day')
    days_index = ema_user.index
    previous_ema_items = []
    for day in days_index:
        previous_day = day - timedelta(days=NUM_DAY_MINUS)
        if previous_day in ema_user.index:
            previous_ema_items.append(ema_user.loc[previous_day][EMA_ITEM])
        else:
            previous_ema_items.append(float('nan'))
#             previous_ema_items.append(ema['ema_score'].mean())
    ema_user['prev_' + EMA_ITEM] = previous_ema_items
    ema_user = ema_user.reset_index()
    new_ema = pd.concat([new_ema, ema_user])
new_ema = new_ema.drop(['index'], axis=1)

70


/homes/gws/joyhe/.local/lib/python3.6/site-packages/ipykernel_launcher.py:93: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [69]:
new_ema.to_csv(r'table_ema_'+str(NUM_DAYS_USED)+'_'+str(NUM_DAYS_BLOCKED)+'_prev'+str(NUM_DAY_MINUS)+'.csv')

In [10]:
def get_filtered_ema_from_table(NUM_GOOD_DAYS_REQUIRED, current_ema):
    '''
    only keeps ema data with strictly more than "NUM_GOOD_DAYS_REQUIRED" days of behavioral data 
    for all activities
    '''
    table0 = current_ema
    table0_filtered = table0.loc[table0.calllog_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.sms_in_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.sms_out_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.unlock_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.conver_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.sleep_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.act_invehicle_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.act_onbike_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.act_onfoot_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.act_still_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.act_tilting_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.act_unknown_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.light_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.audio_amp_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.audio_noise_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.audio_silence_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    table0_filtered = table0_filtered.loc[table0_filtered.audio_voice_num_good_days >= NUM_GOOD_DAYS_REQUIRED]
    new_ema = table0_filtered
    return new_ema

In [11]:
FILE = 'table_ema_'+str(NUM_DAYS_USED)+'_'+str(NUM_DAYS_BLOCKED)+'_prev'+str(NUM_DAY_MINUS)+'.csv'
ema = pd.read_csv(FILE)
if 'Unnamed: 0' in ema.columns:
    ema = ema.drop(['Unnamed: 0'], axis=1)
ema = get_filtered_ema_from_table(7, ema)
len(ema)

779

In [12]:
EMA_ITEM = 'ema_score' # what we are predicting
NUM_TEST_PER_USER = 10

ema['day'] = pd.to_datetime(ema['day'])
ema = ema.set_index('day')
ema = ema.sort_index()
ema = ema.reset_index()
ema_counts = ema.groupby('eureka_id').count()
MIN_USER_DATA = 25
ema_counts = ema_counts.loc[ema_counts.day > MIN_USER_DATA]
valid_users = ema_counts.index
all_users = ema['eureka_id'].drop_duplicates()
ema = ema.set_index('eureka_id')
for u in all_users:
    if u not in valid_users:
        ema = ema.drop(u)
ema = ema.reset_index()
print('users included:',len(ema.groupby('eureka_id').count()))
POPULATION_MEAN = ema[EMA_ITEM].mean()
print('population mean is:', POPULATION_MEAN)

print('ema before drop nan', len(ema))
ema = ema.dropna(subset=['prev_' + EMA_ITEM])
print('ema before drop nan', len(ema))

users included: 14
population mean is: -7.696107784431137
ema before drop nan 668
ema before drop nan 655


# Correlations

In [13]:
filtered_ema = get_filtered_ema_from_table(7, ema)
print(len(filtered_ema))
print('users included:',len(filtered_ema.groupby('eureka_id').count()))

655
users included: 14


In [16]:
corr_table = ema.corr(method ='pearson') 
row_index = [r for r in corr_table.index if "ema" in r]
col_index = [c for c in corr_table.columns if 'ema' not in c]
corr_table_small = corr_table.loc[row_index, col_index]
p_table = get_data.get_p_values(ema)
p_table_small = p_table.loc[row_index, col_index]

## correlations between stability index and ema 

In [17]:
row_index = [r for r in corr_table.index if "ema" in r]
col_index_med_mean = [c for c in corr_table.columns if 'med_mean' in c and 'raw' not in c]
corr_table_small_med_mean = corr_table.loc[row_index, col_index_med_mean]
# print r in desceding order for each ema score
ordered_list_all = pd.DataFrame()
for i in corr_table_small_med_mean.index:
#     print(i)
    ordered_list = corr_table_small_med_mean.loc[i].sort_values(ascending=False).rename('diff')
    ordered_list = ordered_list.to_frame()
    ordered_list = ordered_list.reset_index()
#     print(ordered_list)
    means = []
    stds = []
    num_days = []
    pvals_diff = []
    pvals_mean = []
    pvals_std = []
    pvals_num_days = []
    for item in ordered_list['index']:
        pvals_diff.append(p_table.loc[i][item])
        for b in range(len(behavior_names)):            
            if behavior_names[b] in item:
                means.append(corr_table_small.loc[i][behavior_names[b] + '_avg'])
                stds.append(corr_table_small.loc[i][behavior_names[b] + '_std'])
                num_days.append(corr_table_small.loc[i][behavior_names[b] + '_num_good_days'])
                pvals_mean.append(p_table.loc[i][behavior_names[b] + '_avg'])
                pvals_std.append(p_table.loc[i][behavior_names[b] + '_std'])
                pvals_num_days.append(p_table.loc[i][behavior_names[b] + '_num_good_days'])

    ordered_list['mean'] = means
    ordered_list['std'] = stds                                     
    ordered_list['p_diff'] = pvals_diff
    ordered_list['p_mean'] = pvals_mean
    ordered_list['p_std'] = pvals_std                                     
    ordered_list['ema_item'] = [i] * len(ordered_list)
    ordered_list_all = pd.concat([ordered_list_all, ordered_list])

ordered_list_all 

,index,diff,mean,std,p_diff,p_mean,p_std,ema_item
0,unlock_med_mean,0.393254,0.438417,0.218021,1.184982e-25,3.826472e-32,1.729414e-08,ema_CALM
1,sms_in_med_mean,0.276026,0.282148,0.273239,6.432375e-13,1.877847e-13,1.115218e-12,ema_CALM
2,sms_out_med_mean,0.275536,0.295361,0.271106,7.088203e-13,1.185818e-14,1.692229e-12,ema_CALM
3,audio_silence_med_mean,0.194902,-0.025735,0.146164,4.985427e-07,5.108636e-01,1.741670e-04,ema_CALM
4,sms_out_med_mean_normal,0.194105,0.295361,0.271106,5.559770e-07,1.185818e-14,1.692229e-12,ema_CALM
5,audio_noise_med_mean,0.193023,0.023656,0.159291,6.441859e-07,5.456055e-01,4.219537e-05,ema_CALM
6,act_onfoot_med_mean,0.192475,0.153333,0.224551,6.938885e-07,8.145646e-05,6.235704e-09,ema_CALM
7,unlock_med_mean_normal,0.162157,0.438417,0.218021,3.048933e-05,3.826472e-32,1.729414e-08,ema_CALM
8,calllog_med_mean_normal,0.148711,0.139176,0.055757,1.334753e-04,3.535645e-04,1.540486e-01,ema_CALM
9,sms_in_med_mean_normal,0.119935,0.282148,0.273239,2.106909e-03,1.877847e-13,1.115218e-12,ema_CALM


In [18]:
# Bonferroni correction 
ordered_list_all['p_diff'] = ordered_list_all['p_diff']*17

## correlations between stability index and mean

In [19]:
corr_mean = pd.DataFrame()
absolutes = []
normals = []
absolute_normals = []
pvals_absolute = []
pvals_normal = []
pvals_absolute_normals = []
for behavior in behavior_names:
    absolutes.append(corr_table.loc[behavior + '_med_mean', behavior + '_avg'])
    normals.append(corr_table.loc[behavior + '_med_mean_normal', behavior + '_avg'])
    absolute_normals.append(corr_table.loc[behavior + '_med_mean', behavior + '_med_mean_normal'])
    pvals_absolute.append(p_table.loc[behavior + '_med_mean', behavior + '_avg'])
    pvals_normal.append(p_table.loc[behavior + '_med_mean_normal', behavior + '_avg'])
    pvals_absolute_normals.append(p_table.loc[behavior + '_med_mean', behavior + '_med_mean_normal'])
corr_mean['behavior'] = behavior_names
corr_mean['abs_diff&mean'] = absolutes
corr_mean['normal_diff&mean'] = normals
corr_mean['abs&normal'] = absolute_normals

corr_mean['p_abs'] = pvals_absolute
corr_mean['p_normal'] = pvals_normal

corr_mean['p_abs_normal'] = pvals_absolute_normals

corr_mean = corr_mean.set_index('behavior')

corr_mean

,abs_diff&mean,normal_diff&mean,abs&normal,p_abs,p_normal,p_abs_normal
behavior,,,,,,
calllog,0.916623,0.231471,0.164982,3.181376e-262,2.043396e-09,2.201432e-05
sms_in,0.944311,0.387486,0.281945,1.971455e-317,6.815077e-25,1.957046e-13
sms_out,0.952715,0.362237,0.269413,0.000000e+00,9.677194e-22,2.349806e-12
unlock,0.839042,0.334112,0.030700,8.927907e-175,1.517238e-18,4.328125e-01
conver,0.739167,0.180785,-0.279585,3.458301e-114,3.217781e-06,3.156254e-13
sleep,0.124040,0.201233,-0.798555,1.468835e-03,2.063388e-07,4.279013e-146
act_invehicle,0.762415,0.265414,0.013914,1.482101e-125,5.057070e-12,7.222690e-01
act_onbike,0.776668,-0.043357,-0.109537,3.406742e-133,2.678499e-01,5.008825e-03
act_onfoot,0.869794,-0.024943,-0.201472,1.667762e-202,5.239696e-01,1.994810e-07


In [20]:
corr_mean['normal_diff&mean'].mean()

0.3044148740993883

In [21]:
corr_mean['abs_diff&mean'].mean()

0.6454916291102034